### Install required packages

In [1]:
# install required packages
!pip -q install transformers
!pip install gdown
!pip install "nltk==3.4.5"
!pip install pytorch_pretrained_bert 
!pip install fire

     |████████████████████████████████| 1.4MB 9.1MB/s 
     |████████████████████████████████| 890kB 28.9MB/s 
     |████████████████████████████████| 2.9MB 22.3MB/s 
     |████████████████████████████████| 1.5MB 8.9MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=886a580cfd9fe2315d21f3e9df82abab6975d6ef4f5869a2e9afcb8f21fd9925
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


# COVID-19 Doctor Chatbot - Transformer

## Set up Drive 

In [18]:
# transformer model
!gdown https://drive.google.com/uc?id=1YrV42EAxggTYPDqyG4OylXcZsICailSa
# saved weights
!gdown https://drive.google.com/uc?id=1Km0FXuWXGlSHTjVdTp2a51Up4U1iT7RH
# test data
!gdown https://drive.google.com/uc?id=1rZt4uAKvbjFOYE3lGlUeUm2GJtkH_7P7

Downloading...
From: https://drive.google.com/uc?id=1YrV42EAxggTYPDqyG4OylXcZsICailSa
To: /content/transformers_model.py
100% 1.29k/1.29k [00:00<00:00, 2.03MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Km0FXuWXGlSHTjVdTp2a51Up4U1iT7RH
To: /content/best_model.pth
419MB [00:01, 216MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rZt4uAKvbjFOYE3lGlUeUm2GJtkH_7P7
To: /content/test_data.pth
100% 1.07M/1.07M [00:00<00:00, 68.2MB/s]


In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from transformers_model import transformers_model
from pytorch_pretrained_bert import BertTokenizer

import fire
import json
from collections import defaultdict

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist
from nltk.util import ngrams

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Defined evaluation metrics

In [25]:
def bleu(predict, target, n):
    return sentence_bleu([target], predict, weights=tuple(1 / n for i in range(n)))


def nist(predict, target, n):
    if len(predict) < n or len(target) < n:
        return 0
    return sentence_nist([target], predict, n)


def cal_entropy(generated):
    etp_score = [0.0, 0.0, 0.0, 0.0]
    div_score = [0.0, 0.0, 0.0, 0.0]
    counter = [defaultdict(int), defaultdict(int),
               defaultdict(int), defaultdict(int)]
    for gg in generated:
        g = gg.rstrip().split()
        for n in range(4):
            for idx in range(len(g)-n):
                ngram = ' '.join(g[idx:idx+n+1])
                counter[n][ngram] += 1
    for n in range(4):
        total = sum(counter[n].values()) + 1e-10
        for v in counter[n].values():
            etp_score[n] += - (v+0.0) / total * (np.log(v+0.0) - np.log(total))
        div_score[n] = (len(counter[n].values())+0.0) / total
    return etp_score, div_score


def cal_length(sentences):
    sen_length = [len(s.split()) for s in sentences]
    return np.mean(sen_length), np.var(sen_length)


def calculate_metrics(predict, reference):
    reference_len = len(reference)
    predict_len = len(predict)

    #-------------------bleu----------
    bleu_2 = bleu(predict, reference, 2)
    bleu_4 = bleu(predict, reference, 4)
    #-------------------nist----------
    nist_2 = nist(predict, reference, 2)
    nist_4 = nist(predict, reference, 4)
    #-------------------meteor----------
    predict = " ".join(predict)
    reference = " ".join(reference)
    meteor_scores = meteor_score([reference], predict)
    return bleu_2, bleu_4, nist_2, nist_4, meteor_scores


def top_k_logits(logits, k):
    """Mask logits so that only top-k logits remain
    """
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1].unsqueeze(1).repeat(1, logits.shape[-1])
    return torch.where(logits < min_values, torch.ones_like(logits, dtype=logits.dtype) * -1e10, logits)



In [26]:
top_k = 50
temperature = 1.0
decoder_path='best_model.pth'
gpu_id=0

## Load pre-trained model and test set


In [27]:
# load model
print('load the model....')
device = torch.device(f"cuda:{gpu_id}")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = transformers_model()
model.load_state_dict(torch.load(decoder_path))

device = torch.device(f"cuda:0")
model.to(device)
model.eval()

print('load success')

load the model....


100%|██████████| 231508/231508 [00:00<00:00, 1058185.04B/s]


load success


In [29]:
# load test data
test_data = torch.load("test_data.pth")
test_dataset = TensorDataset(*test_data)
test_dataloader = DataLoader(dataset=test_dataset, shuffle=False, batch_size=1)

## generate output samples

In [54]:
# start generate samples
update_count = 0

bleu_2scores = 0
bleu_4scores = 0
nist_2scores = 0
nist_4scores = 0

meteor_scores = 0
sentences = []
print('start generating....')

for batch in test_dataloader:
        with torch.no_grad():
            batch = [item.to(device) for item in batch]
            encoder_input, decoder_input, mask_encoder_input, _ = batch
            #past, _ = model.encoder(encoder_input, mask_encoder_input)
            past = model.encoder(encoder_input, mask_encoder_input)[0]

            prev_pred = decoder_input[:, :1]
            sentence = prev_pred

            # decoding loop
            for i in range(100):
              #logits, _ = model.decoder(sentence, encoder_hidden_states=past)
              logits = model.decoder(sentence, encoder_hidden_states=past)[0]
              logits = model.linear(logits)
              logits = logits[:, -1]
              logits = logits.squeeze(1) / temperature
              
              logits = top_k_logits(logits, k=top_k)
              probs = F.softmax(logits, dim=-1)
              prev_pred = torch.multinomial(probs, num_samples=1)
              sentence= torch.cat([sentence, prev_pred], dim=-1)
              if prev_pred[0][0] == 102:
                  break

            predict = tokenizer.convert_ids_to_tokens(sentence[0].tolist())

            encoder_input = encoder_input.squeeze(dim=0)
            encoder_input_num = (encoder_input != 0).sum()
            inputs = tokenizer.convert_ids_to_tokens(encoder_input[:encoder_input_num].tolist())

            decoder_input = decoder_input.squeeze(dim=0)
            decoder_input_num = (decoder_input != 0).sum()

            reference = tokenizer.convert_ids_to_tokens(decoder_input[:decoder_input_num].tolist())
            #print(calculate_metrics(predict[1:-1], reference[1:-1]))
            temp_bleu_2, temp_bleu_4, temp_nist_2, temp_nist_4, temp_meteor_scores = calculate_metrics(predict[1:-1], reference[1:-1])
            

            bleu_2scores += temp_bleu_2
            bleu_4scores += temp_bleu_4
            nist_2scores += temp_nist_2
            nist_4scores += temp_nist_4

            meteor_scores += temp_meteor_scores
            sentences.append(" ".join(predict[1:-1]))
            
            # print some samples
            if update_count % 50 == 0:
                patient = ' '.join(inputs[1:-1])
                reference = ' '.join(reference[1:-1])
                predict = ' '.join(predict[1:-1])
                
                print('-'*20 + f"example {update_count}" + '-'*20)
                print(f"Patient: {patient.replace(' ##','')}")
                print(f"Reference: {reference.replace(' ##','')}")
                print(f"Predict: {predict.replace(' ##','')}")
            
            update_count += 1

start generating....
--------------------example 0--------------------
Patient: i have had mild chest pain for over a week . it now seems more persistent and pronounced . i don ' t have shortness of breath or any other covid - 19 symptoms , except some fatigue . i have been traveling a lot in high risk areas . should i get tested ?
Reference: brief opinion : yes i would advise screening due to your exposure . fever is very commonly associated with covid - 19 . stay at home , rest , drink fluids and monitor your temperature . arrange the testing which also may include a chest image with your pcp . since your have been traveling , a pulmonary embolism is another possible cause of your shortness of breath . would you like to video or text chat with me ?
Predict: brief opinion : and symptoms you are not no your .


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

--------------------example 50--------------------
Patient: will warm weather stop the outbreak of covid - 19 ?
Reference: it is not yet known whether weather and temperature affect the spread of covid - 19 . some other viruses , like those that cause the common cold and flu , spread more during cold weather months but that does not mean it is impossible to become sick with these viruses during other months . there is much more to learn about the transmissibility , severity , and other features associated with covid - 19 and investigations are ongoing .
Predict: brief opinion : the home and it . would you like to would you may so , fever video or a person with me ?
--------------------example 100--------------------
Patient: hello , i have a quesition for you . is it safe to use public transit ? [SEP] if you must use public transit , regularly and thoroughly wash your hands and avoid touching your face with unwashed hands . continue to distance yourself 2 metres ( 6 feet ) away from ot

In [55]:
entro, dist = cal_entropy(sentences)
mean_len, var_len = cal_length(sentences)
print(f'avg: {mean_len}, var: {var_len}')
print(f'entro: {entro}')
print(f'dist: {dist}')
print(f'test bleu_2scores: {bleu_2scores / update_count}')
print(f'test bleu_4scores: {bleu_4scores / update_count}')
print(f'test nist_2scores: {nist_2scores / update_count}')
print(f'test nist_4scores: {nist_4scores / update_count}')
print(f'test meteor_scores: {meteor_scores / update_count}')

avg: 32.30827067669173, var: 367.38617219741076
entro: [4.1226581566545, 6.168883457345389, 7.250598482876699, 7.724759428687033]
dist: [0.03164998836397413, 0.2939481268011457, 0.6407839245844544, 0.8073370959466186]
test bleu_2scores: 0.02914540023222242
test bleu_4scores: 0.009063939642336982
test nist_2scores: 0.28146148716215974
test nist_4scores: 0.2809897102343668
test meteor_scores: 0.09932598001363399


# COVID-19 Doctor Chatbot - GPT2



## Set up drive

In [3]:
!gdown https://drive.google.com/uc?id=1-Cmprk6ZOvSrOCxoZldu-qCXuPGyFjtz

Downloading...
From: https://drive.google.com/uc?id=1-Cmprk6ZOvSrOCxoZldu-qCXuPGyFjtz
To: /content/pytorch_model.bin
510MB [00:03, 147MB/s]


In [ ]:
!mv pytorch_model.bin output-small-save/

## Dataset setup

Set up dataset and view first 5 items in training and validation sets

In [ ]:
import dataset

df_trn, df_val = dataset.make_dataset(path_to_train_data="data/train_data.json", path_to_validation_data="data/validate_data.json")

> /content/drive/My Drive/Colab/GPT2-finetune/dataset.py(12)make_dataset()
-> f_validate.close()
(Pdb) c


In [ ]:
df_trn.head(5)

,response,context
0,"Hello, I understand your concern. I just have ...","Hello doctor, I get a cough for the last few d..."
1,"Hello, I can understand your concern.In my opi...","Hello doctor, I am suffering from coughing, th..."
2,Hello. Anxiety can manifest itself in physical...,"Hello doctor,I am a 23-year-old man. I have an..."
3,"Hello,please answer the following:Any travel h...","Hello doctor,Last night I was getting chills, ..."
4,Hello and welcome to Ask A Doctor service.I ha...,"Hi, I am Chaitanya, 27 years old. I use to swi..."


In [ ]:
df_val.head(5)

,response,context
0,Corona-virus. At 33 you may not need testing. ...,I have a constant cough and my chest has now b...
1,Less likely. Recommended to stay 6 feet apart....,If someone has carona virus and iam passing by...
2,"Test Please stay at home, rest, drink fluids...",I am concerned that I’m showing symptoms of co...
3,Death. At your age the risk of death is the fo...,What are my chances of becoming seriously ill ...
4,Unknown but low Based on current data it is ...,Nervous about coronavirus. I am 26 years old a...


## Evaluating


In [ ]:
!python main.py

2020-12-04 22:00:10.930707: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
12/04/2020 22:00:12 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
12/04/2020 22:00:23 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f959d6c1160>
12/04/2020 22:00:23 - INFO - __main__ -   Evaluate the following checkpoints: ['output-small-save']
12/04/2020 22:00:29 - INFO - utils -   Creating features from dataset file at cached
12/04/2020 22:00:29 - INFO - utils -   Sav

### Show up to 5 test results

In [ ]:
test_chatbot = []

for i in range(5): #len(test_query)
  tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
  model = AutoModelWithLMHead.from_pretrained('output-small-save')
  # append the new user input tokens to the chat history
  bot_input_ids = tokenizer.encode(test_query[i] + tokenizer.eos_token, return_tensors='pt')
  print("User: {} \n".format(test_query[i]))

  # generated a response while limiting the total chat history to 1000 tokens, 
  chat_history_ids = model.generate(
      bot_input_ids, max_length=100,
      pad_token_id=tokenizer.eos_token_id,  
      no_repeat_ngram_size=3,       
      do_sample=True, 
      top_k=10, 
      top_p=0.7,
      temperature = 0.8
  )

  # pretty print last ouput tokens from bot
  print("Chatbot: {} \n\n".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
  test_chatbot.append(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))

print(len(test_chatbot))


/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


User: I have all the symptoms except fever, I went to Medicross and Dr said I can get tested if I want to I'm not sure if I should. She gave me antibiotics Klacid XL 500mg, she said I can take it if I feel worse I'm worried it will make immune system bad? 

Chatbot: Take a test.    You will likely need to take the test. 


User: I have pain/discomfort in my lungs. I don't experience simultaneous on both lungs and it not always at the hame position. I don't have a head nor do I have high temperature. I sneeze and cough maybe once a day. Do I have corona, should I get tested? 

Chatbot: I can understand your concern. I have no idea what you are experiencing. If you have a cold or other respiratory disease, consult your local physician. 


User: I travelled to Mauritius and do not have symptoms. Should I get tested for covid19? 

Chatbot: Yes, if you are in contact with someone with Covid19. You should be tested for Covid 19.    If you have a fever and or fever, get tested and get tested.

## Generate test results


In [ ]:
with open('gpt2-results.txt') as f:
    test_chatbot = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
test_chatbot = [x.strip() for x in content] 

## Metrics

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.translate.nist_score import sentence_nist

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import metrics
bleu_2, bleu_4, meteor, nist_2, nist_4 = metrics.get_metrics(test_chatbot, test_response)

In [ ]:
 bleu_2, bleu_4, meteor, nist_2, nist_4

(0.1987791535170549,
 0.10355670162282384,
 0.22840520757502006,
 0.9900288417339577,
 1.0254752183668514)

## Interactive Chat

A variety of methods can be used in responces generation. You can find more details about these methods by this [link](https://huggingface.co/blog/how-to-generate). 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('output-small-save')

# Let's chat for 1 line
for step in range(1):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=100,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=10, 
        top_p=0.7,
        temperature = 0.8
    )
    
    # pretty print last ouput tokens from bot
    print("Chatbot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:852: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


>> User:Hi I am 39 years old and returned from Germany 19 days ago. Yesterday I started getting a sore throat, runny nose. Today I have sinus pressure and a headache with a blocked nose, throat seems to be improving. Should I get tested. If so how?
Chatbot: Hello,    If you have a sore nose, you should definitely get tested for COVID-19.   Would you like to video or text chat with me?


# Training BART with Google Colab

- This is our third and final model.
- It's using Hugging Face based code.
- There are over 400m parameters, so you are not going to be able to train it without using COLAB.
- Please ensure that you have nltk version above 3.2 

In [ ]:
!pip install simpletransformers -q
!pip install nltk==3.4.5

import nltk
print(nltk.__version__)

try:
  from nltk.translate.meteor_score import meteor_score
  print('Meteor score will not work without the right ntlk version')
except ImportError:
  print('Still import issue')

In [ ]:
import tqdm
import spacy
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from simpletransformers.seq2seq import Seq2SeqModel
from sklearn.model_selection import train_test_split
import os
import os.path
import tensorflow as tf
import os
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.nist_score import sentence_nist

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('good to go')
print(f'Device: {DEVICE}')

csv_link = 'https://raw.githubusercontent.com/chophilip21/covid_dialogue/main/dialogue.csv' #original
augmented_link = 'https://raw.githubusercontent.com/chophilip21/covid_dialogue/main/augmented.csv' #augmented version
txt_link = 'https://raw.githubusercontent.com/chophilip21/covid_dialogue/main/covid_additional.txt' # raw text version

dataset = pd.read_csv(augmented_link, names = ['input_text', 'target_text'], header=0)
train_df, test_df = train_test_split(dataset, test_size=0.2)
valid_df, test_df = train_test_split(test_df, test_size=0.5)

print('The length of train_df is: ', len(train_df))
print('The length of valid is: ', len(valid_df))
print('The length of test_df is: ', len(test_df))


def txt_to_dict(txt_path, save_path):

    patient = []
    doctor = []

    with open(txt_path, 'r') as f:
        lines = f.readlines()

        for i, line in enumerate(lines):
            if line.startswith('Patient:'): 
                patient.append(' '.join(lines[i+1:i+2]))
            
            elif line.startswith('Doctor:'):
                doctor.append(' '.join(lines[i+1: i+2]))

    data = {'src': patient, 'trg': doctor}

    return data


In [ ]:
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 50,
    "train_batch_size": 4, # check if we can have bigger weights. 
    "eval_batch_size": 1,
    "output_dir": 'weights',
    "num_train_epochs": 5,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "evaluate_during_training": True,
    "evaluate_generated_text": True,
    "evaluate_during_training_verbose": True,
    "use_multiprocessing": True,
    "gradient_accumulation_steps": 1,
    "max_length": 50,
    "manual_seed": 4,
}


model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,
)


model.train_model(train_df, eval_data=valid_df)

def save_prediction(labels, preds):

  for prediction in preds:
    with open('prediction.txt', 'a') as f:
      f.write(prediction + '\n')

  print('complete')



print(model.eval_model(test_df, save = save_prediction))

"""
Predictions on a random string
"""

test = "Hi doctor, What are the symptoms of Covid-19..?"
inference = model.predict([test])
print(inference)

In [ ]:
nltk.download('wordnet')


"""
The code crashes when I evaluate within the evalulate function. 
-This is an alternative
"""

def nist_2(labels, preds):

    label = ' '.join([str(elem) for elem in labels])
    prediction = ' '.join([str(elem) for elem in preds])
  
    if len(prediction) < 2 or len(label) < 2:
        return 0
    return sentence_nist([label], prediction, 2)

def nist_4(labels, preds):

    label = ' '.join([str(elem) for elem in labels])
    prediction = ' '.join([str(elem) for elem in preds])

    if len(prediction) < 4 or len(label) < 4:
        return 0

    return sentence_nist([label], prediction, 4)

def calculate_m_score(target, predictions, length):

  score = 0

  for t, p in zip(target, predictions):
    score += meteor_score(t, p)

  
  return score / length


predictions = []

with open('prediction.txt') as fp:
  line = fp.readline()
  line = line.strip()
  
  while line:
    if len(line) > 1:
      predictions.append(line)
    line = fp.readline()

# label = ' '.join([str(elem) for elem in labels])
pred = ' '.join([str(elem) for elem in predictions])
target = test_df.target_text

bleu2 = sentence_bleu(target, pred, weights=tuple(1 / 2 for i in range(2)))
bleu4 = sentence_bleu(target, pred, weights=tuple(1 / 4 for i in range(2)))
nist2 = nist_2(target, pred)
nist4 = nist_4(target, pred)


print('bleu2 is {}'.format(bleu2))
print('bleu4 is {}'.format(bleu4))
print('nist2 is {}'.format(nist2))
print('nist4 is {}'.format(nist4))

meteor = calculate_m_score(target, predictions, 71)

print('meteor is {}'.format(meteor))